
# Projet Maison 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
GEO = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
GEO[['lat','lon']]=GEO['geo_point_2d'].str.extract("(.*),(.*)").astype(float)
GEO['cp_ville']=GEO['Code Postal']+" "+GEO['Commune']

# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes
# - cp_ville : Code Postal + " " + "Commune"

**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [3]:
# fonction recherche de ville
def search_city(lat, long):
    min_lat = GEO['lat']-lat
    min_long = GEO['lon']-long
    d_index =np.sqrt(min_lat**2+min_long**2).idxmin()
    ville = GEO['cp_ville'].loc[d_index]
    return ville
    pass

In [4]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    res = deg+mn/60+sec/3600
    return res
    pass

In [5]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)

search_city(lat, lon)

'71330 BOSJEAN'

In [6]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

In [7]:
GEO['cp_ville'].iloc[4]

'84310 MORIERES-LES-AVIGNON'

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [8]:
# GEO_SHAPE

# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
import json
data_json = GEO['geo_shape'].apply(json.loads)
GEO_SHAPE = pd.Series(data_json)

In [9]:
GEO_SHAPE.iloc[5]['coordinates']

[[[1.849459801983121, 50.17500092547019],
  [1.819671090612144, 50.17748470237378],
  [1.816161558052287, 50.17984888796438],
  [1.813013582655742, 50.182288307558636],
  [1.816809990568512, 50.19761032874879],
  [1.817714156165066, 50.19783456129657],
  [1.837638053630789, 50.19473749649713],
  [1.850682486289417, 50.199413306477915],
  [1.856660811234537, 50.19426918491142],
  [1.852186532901586, 50.178717587847764],
  [1.861963155005413, 50.177434384198754],
  [1.849459801983121, 50.17500092547019]]]

In [10]:
# value_counts des valeurs "type"
def get_types():
    res = GEO_SHAPE.apply(lambda x: x['type']).value_counts()
    return res
    pass

In [11]:
# value_counts des longueurs de "coordinates"
def get_coordinates_len():
    res = GEO_SHAPE.apply(lambda x: len(x['coordinates'])).value_counts()
    return res
    pass

In [12]:
# commune constituée du plus grand nombre de polygones
def get_most_complex_city():
    loc_max = GEO_SHAPE.apply(lambda x: len(x['coordinates'])).idxmax()
    res = GEO['cp_ville'].loc[loc_max]
    return res
    pass

In [13]:
# commune constituée du plus grand nombre de polygones
def get_nb_cities_2_polygons():
    ville_chercher = GEO_SHAPE.apply(lambda x: 1 if (x['type']=='Polygon' and len(x['coordinates'])==2) else 0)
    res = ville_chercher.sum()
    return res
    pass

In [14]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)
       
    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [15]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ok
test_partie_B2 (__main__.Session2Test) ... ok
test_partie_B3 (__main__.Session2Test) ... ok
test_partie_B4 (__main__.Session2Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.083s

OK
